# Grouping and aggregating spark dataframes

In [1]:
from pyspark.sql import SparkSession
from pathlib import Path
data_dir = Path('../pyspark/Python-and-Spark-for-Big-Data-master/Spark_DataFrames')

In [2]:
spark = SparkSession.builder.appName('aggs').getOrCreate()

21/07/19 08:04:12 WARN Utils: Your hostname, GBLON1WLZ13699 resolves to a loopback address: 127.0.1.1; using 10.164.85.96 instead (on interface eth2)
21/07/19 08:04:12 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
21/07/19 08:04:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
data_path = str(data_dir/'sales_info.csv')

In [4]:
df = spark.read.csv(data_path, inferSchema=True, header=True)

In [5]:
df.show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|    Sam|200.0|
|   GOOG|Charlie|120.0|
|   GOOG|  Frank|340.0|
|   MSFT|   Tina|600.0|
|   MSFT|    Amy|124.0|
|   MSFT|Vanessa|243.0|
|     FB|   Carl|870.0|
|     FB|  Sarah|350.0|
|   APPL|   John|250.0|
|   APPL|  Linda|130.0|
|   APPL|   Mike|750.0|
|   APPL|  Chris|350.0|
+-------+-------+-----+



In [6]:
df.printSchema()

root
 |-- Company: string (nullable = true)
 |-- Person: string (nullable = true)
 |-- Sales: double (nullable = true)



In [7]:
df.groupBy('Company').mean().show()

+-------+-----------------+
|Company|       avg(Sales)|
+-------+-----------------+
|   APPL|            370.0|
|   GOOG|            220.0|
|     FB|            610.0|
|   MSFT|322.3333333333333|
+-------+-----------------+



Aggregate without grouping. Takes a dict of columns and operations

In [8]:
df.agg({'Sales':'max'}).show()

+----------+
|max(Sales)|
+----------+
|     870.0|
+----------+



We can also use this dict syntax for grouping too

In [9]:
group_data = df.groupBy('Company')

In [10]:
group_data.agg({'Sales': 'max'}).show()

+-------+----------+
|Company|max(Sales)|
+-------+----------+
|   APPL|     750.0|
|   GOOG|     340.0|
|     FB|     870.0|
|   MSFT|     600.0|
+-------+----------+



## Using pyspark functions

In [11]:
from pyspark.sql.functions import countDistinct, avg, stddev

We import aggregation functions and pass them into `select`. We can also use aliases to rename the output.

In [13]:
df.select(countDistinct('Sales').alias('Average Sales')).show()

+-------------+
|Average Sales|
+-------------+
|           11|
+-------------+



In [15]:
df.select(stddev('Sales')).show()

+------------------+
|stddev_samp(Sales)|
+------------------+
|250.08742410799007|
+------------------+



In [17]:
from pyspark.sql.functions import format_number

In [18]:
sales_std = df.select(stddev('Sales').alias('std'))

In [19]:
sales_std.show()

+------------------+
|               std|
+------------------+
|250.08742410799007|
+------------------+



In [21]:
sales_std.select(format_number('std', 2).alias('std')).show()

+------+
|   std|
+------+
|250.09|
+------+



Ordering

In [25]:
df.orderBy('Sales', ascending=False).show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|     FB|   Carl|870.0|
|   APPL|   Mike|750.0|
|   MSFT|   Tina|600.0|
|     FB|  Sarah|350.0|
|   APPL|  Chris|350.0|
|   GOOG|  Frank|340.0|
|   APPL|   John|250.0|
|   MSFT|Vanessa|243.0|
|   GOOG|    Sam|200.0|
|   APPL|  Linda|130.0|
|   MSFT|    Amy|124.0|
|   GOOG|Charlie|120.0|
+-------+-------+-----+

